# Vault KMIP Secrets Engine for MongoDB Encryption (ADP)

teaser: Help implement the "Assumed Breach" principal of a Zero Trust methodology by leveraging Vault’s KMIP Secret engine to encrypt all MongoDB Enterprise data.

## Overview

The KMIP secrets engine allows Vault to act as a **Key Management Interoperability
Protocol (KMIP)** server provider and handle the lifecycle of its KMIP managed objects.

In this tutorial, we will leverage this KMIP secrets engine for external MongoDB encryption key management.

A few other examples (not included here) are:
* MySQL Enterprise
* VSphere VMs and VSANs
* NetApp
* EMC Unity and PowerEdge/Max (in progress)

This encryption workflow protects against the scenario where an adversary has gained privileged access to the Linux/Windows box running the database. In this case, they may only have permissions to inspect database files on the host and are not actually logged into the database itself.

## Prerequisites

Install MongoDB client. Here is an example for Mac

In [ ]:
brew tap mongodb/brew
brew install mongodb-community@4.4

## Test MongoDB unecrypted

teaser: In this challenge you will see the impact of a breach with unencrypted database files

**notes:**
In recent news, it seems as though a new data breach occurs every other week...

With the advent of events like the SolarWinds hack (a supply chain attack), it is fair to say that we must always assume adversaries are on our network.
They are likely working to escalate credentials and expand their footprint laterally. As a last line of defense, we must ensure our customer data is encrypted!

In this challenge, we will role play an attacker who was able to gain remote access to our database machine.

They do not have priviledged database credentials, only RDP/SSH access to the machine.
You will simulate exfiltrating data from unencrypted MongoDB storage files.

### assignment:

Welcome to the lab. 

Before we introduce Vault for filesystem encryption. Lets test a write to MongoDB.

#### Start MongoDB Server - unencrypted

Start MongoD server (without encryption enabled). The process will not exit.

In [ ]:
docker run --rm --name mongodb -d \
    -p 27017:27017 \
    mongo --bind_ip_all

Insert an example record

In [ ]:
mongo --eval 'db.examples.insertOne(
    { name: "sue", age: 26 }
)' && sleep 30

Now `cat` out the mongodb collection file, there should be several. It will likely be the "`7`th" file.

**NOTE:** It cloud take about two minutes for the full contents to be written to disk. There will be multiple lines of data.

In [ ]:
docker exec -it mongodb sh -c "cat /data/db/collection-7*.wt"

Sample Output
```shell
:�7�Knamesueage:@4*r/�8���䁜�H2
V�F���������.֖�T�������/��`encryption=(keyid=,name=),block_metadata_encrypted=false,block_metadata=[access_pattern_hint=none,allocation_size=4KB,app_metadata=(formatVersion=1),assert=(commit_timestamp=none,durable_timestamp=none,read_timestamp=none),block_allocation=best,block_compressor=snappy,cache_resident=false,checkpoint=,checkpoint_backup_info=,checkpoint_lsn=,checksum=on,collator=,columns=,dictionary=0,encryption=(keyid=,name=),<snip>
```

The contents and metadata are in clear text on disk. Notice the plaintext "`namesueage`" and the unecrypted metadata in the collection file.
An adversary in this scenario would only need to gain remote access to the physical database machine in order to exfiltrate critical customer data.

#### Stop MongoDB Server - unencrypted

Cleanup mongodb and move to the next challenge.

In [ ]:
docker stop mongodb && docker rm mongodb

## Configure the Vault KMIP Secrets Engine

teaser: In this challenge you will configure Vault's KMIP Secret Engine for external KMIP object management.

notes:

The KMIP secrets engine allows Vault to act as a **Key Management Interoperability Protocol (KMIP)** server provider and handle the lifecycle of its KMIP managed objects.

KMIP is a standardized protocol that allows services and applications to perform cryptographic operations without having to manage cryptographic material, otherwise known as managed objects, by delegating its storage and lifecycle to a key management server.

In this section we will configure a "`scope`" for managing this application's objects. Within the scope, we designate `roles` that define access control around allowed KMIP operations.

Finally, we will create the certificate and the key for MongoDB to authenticate to Vault's KMIP listener.

### Prerequisites

In [ ]:
WORK_DIR=config/vault/kmip

In [ ]:
mkdir -p $WORK_DIR/ssl

### assignment

#### Start Vault Server

Run the following command. This will start the vault process in the background.

In [ ]:
docker run -d --rm --name vault \
    --cap-add IPC_LOCK \
    -p 8200:8200 -p 5696:5696 \
    -e 'VAULT_DEV_ROOT_TOKEN_ID=root' \
    -e 'VAULT_DEV_LISTEN_ADDRESS=0.0.0.0:8200' \
    hashicorp/vault-enterprise:1.7.5_ent
    # -e "VAULT_ADDR=http://127.0.0.1:8200" \

Configure environment variables for connecting to Vault. We've set the dev mode root token to "`root`".

In [ ]:
export VAULT_ADDR="http://127.0.0.1:8200"
export VAULT_TOKEN=root
# echo "export VAULT_ADDR=$VAULT_ADDR" >> /root/.bashrc

Check vault status

In [ ]:
vault status

#### Enable the KMIP Secrets Engine.

In [ ]:
vault secrets enable kmip

Configure the KMIP Listener (5696 is the standard default port). You can also set key types and lengths.

#### Configure KMIP secrets engine

In [ ]:
vault write kmip/config listen_addrs=0.0.0.0:5696 \
  tls_ca_key_type="rsa" \
  tls_ca_key_bits=2048 \
  server_hostnames="192.168.17.167","localhost"

* `server_hostnames` - Set SAN DNS names for TLS. Click [here](https://www.vaultproject.io/api/secret/kmip#server_hostnames) for info.

Confirm configuration

In [ ]:
vault read kmip/config

#### Create KMIP Scope and Role

* **Scopes** partition KMIP managed objects into multiple named buckets.
* **Roles** are managed within buckets and can be assigned various permitted KMIP operations.

<img src=https://learn.hashicorp.com/img/vault-kmip-2.png width=640>

Create a `hashicups` scope for the app's managed objects.

In [ ]:
# Create a scope
vault write -f kmip/scope/hashicups

List the existing scopes

In [ ]:
vault list kmip/scope

Create a "`payments`" role under the `hashicups` scope that specifies the allowed [KMIP operations](http://docs.oasis-open.org/kmip/spec/v1.4/os/kmip-spec-v1.4-os.html#_Toc490660840) that MongoDB can perform.

In [ ]:
# Create a role
vault write kmip/scope/hashicups/role/payments operation_all=true

* Setting the `operation_all` parameter to `true` - role is allowed to perform all KMIP client-server operations.

Refer to Step 6 to learn how to modify the allowed operation list.

List existing roles under `hashicups` scope.

In [ ]:
vault list kmip/scope/hashicups/role

Save the KMIP CA cert that we will pass to MongoDB.

In [ ]:
vault read -format json kmip/ca | jq -r .data.ca_pem | tee $WORK_DIR/ssl/ca.pem

#### Client certificate generation

These Leaf/CA certs and keys allow MongoDB to authenticate to Vault.

Create the leaf cert and private key. Then save them as a `client.pem`
This cert and key will be used by Mongo to authenticate to Vault.

Generate a certificate in PEM format and save it to a JSON file named `credential.json`

In [ ]:
vault write -format=json \
  kmip/scope/hashicups/role/payments/credential/generate \
  format=pem | tee $WORK_DIR/ssl/credential.json

Extract the certificate from the `credential.json` using `jq` and save it to a file called `cert.pem`.

In [ ]:
jq -r .data.certificate $WORK_DIR/ssl/credential.json | tee $WORK_DIR/ssl/cert.pem

Extract the private key from the `credential.json` using `jq` and save it to a file called `key.pem`.

In [ ]:
jq -r .data.private_key $WORK_DIR/ssl/credential.json | tee $WORK_DIR/ssl/key.pem

#### KMIP client configuration

Combine the `cert.pem` and `key.pem` and save it as `client.pem`.

In [ ]:
cat $WORK_DIR/ssl/{cert.pem,key.pem} | tee $WORK_DIR/ssl/client.pem

Confirm that you see the `client.pem` and `ca.pem` files.

In [ ]:
ls -l $WORK_DIR/ssl/

With the Vault configuration all set, we can now encrypt MongoDB.

## Test MongoDB Encryption via Vault KMIP

teaser: In this challenge you will see the impact of a breach with encrypted database files

notes:

With Vault's KMIP Secret Engine all set up, we can now start MongoDB with KMIP encryption enabled.

We will role play as the same adversary as before, but show that any exfiltrated data would prove useless to the attacker as it is now encrypted ciphertext via Vault!

### assignment

Now, we can start MongoDB with Encryption leveraging Vault as the KMIP Key Management Server.

#### Build Dockerfile

In [ ]:
pushd $WORK_DIR

Set environment variables. Specify your own `DOCKER_USERNAME`.

In [ ]:
export MONGODB_VERSION=4.4
export DOCKER_USERNAME=peterphan

In [ ]:
curl -O --remote-name-all https://raw.githubusercontent.com/docker-library/mongo/master/$MONGODB_VERSION/{Dockerfile,docker-entrypoint.sh}

#### Build mongodb-enterprise container

In [ ]:
chmod 755 ./docker-entrypoint.sh
docker build --build-arg MONGO_PACKAGE=mongodb-enterprise \
    --build-arg MONGO_REPO=repo.mongodb.com \
    -t $DOCKER_USERNAME/mongo-enterprise:$MONGODB_VERSION . > build.log 2>&1

Confirm image was created.

In [ ]:
docker images | head -n 2

Sample Output
```shell
REPOSITORY                                                    TAG             IMAGE ID       CREATED          SIZE
peterphan/mongo-enterprise                                    4.4             268f772c93ea   21 seconds ago   662MB
```

In [ ]:
popd

#### Start MongoDB server - encrypted

Start MongoD server (with encryption enabled).

In [ ]:
docker run --rm --name mongodb -d \
    -p 27017:27017 \
    -v ${PWD}/config/vault/kmip/db:/data/db \
    -v ${PWD}/config/vault/kmip/ssl:/data/ssl \
    $DOCKER_USERNAME/mongo-enterprise:$MONGODB_VERSION \
    --dbpath /data/db \
    --logpath /var/log/mongodb/mongo.log \
    --enableEncryption \
    --kmipServerName 192.168.17.167 \
    --kmipPort 5696 \
    --kmipServerCAFile /data/ssl/ca.pem \
    --kmipClientCertificateFile /data/ssl/client.pem \
    --bind_ip_all

You can verify that MongoDB was able to connect to Vault's KMIP Secret engine with the following command

In [ ]:
docker exec -it mongodb sh -c \
    "cat /var/log/mongodb/mongo.log  | grep -i kmip | jq"

The output should look like this:
```shell
{
  "t": {
    "$date": "2021-04-21T16:07:30.855+00:00"
  },
  "s": "I",
  "c": "STORAGE",
  "id": 24199,
  "ctx": "initandlisten",
  "msg": "Created KMIP key",
  "attr": {
    "keyId": "3ggasHBokpcWjwau4En8uGj6XO091QXL"
  }
}
```

#### Confirm

In [ ]:
mongo --eval 'db.examples.insertOne({ name: "sue", age: 26 })' \
    && sleep 30

Now `cat` out the mongodb collection file, there should be several. It will likely be the "`7`th" file.

**NOTE:** It cloud take about two minutes for the full contents to be written to disk. There will be multiple lines of data.

In [ ]:
docker exec -it mongodb sh -c "cat /data/db/collection-7*.wt" 

Now the contents of the file are encrypted! You should not be able to see any object data or metadata in plaintext.

## Conclusion

A critical principal in implementing Zero Trust is to always assume a breach.
With the implmentation of Vault's KMIP secret engine, we've ensured that our customer data is secure even if your adversaries gain access to physical database hosts.

This tutorial is based on work from Andrew Klaas, Troy Fluegge, Vault Learn guide.

## Resources

* Learn - [KMIP Secrets Engine](https://learn.hashicorp.com/tutorials/vault/kmip-engine)
* Lab - [Vault KMIP Secrets Engine for MongoDB Encryption (ADP)](https://play.instruqt.com/hashicorp/tracks/vault-kmip-secrets-engine-mongodb)

## Clean Up

### Stop Vault and MongoDB

In [ ]:
docker stop mongodb vault
docker rm mongodb vault

In [ ]:
pkill vault mongo

### Remove artifacts

In [ ]:
rm -rf $WORK_DIR

### Remove Docker Image

In [ ]:
docker image rm $DOCKER_USERNAME/mongo-enterprise:4.4